In [1]:
import os
import math
import json
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from datetime import datetime
# from patsy import dmatrices
from patsy import dmatrix
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf

In [2]:
from causalinference import CausalModel
from causalinference.utils import random_data

In [3]:
Data_Root = '/Data/Promotion/'

In [4]:
CIs = {'90': 1.645, '95': 1.96, '99': 2.576}

In [5]:
labels = ['Male', 'Female']

In [6]:
colors = sns.color_palette()[:len(labels)]

In [7]:
reg_data = pd.read_csv(Data_Root+"revision/reg_data_drop_missing.csv", header=0, dtype={'matched_tid': str, \
        'matched_tid_retweet': str, 'matched_tid_original': str, 'pub_year': str})

In [8]:
reg_data = reg_data.loc[reg_data['gender'].isin(['Female', 'Male'])]
reg_data = reg_data.loc[reg_data.affiliation_cate != 'unknown']
reg_data.index = range(len(reg_data))

In [9]:
len(reg_data)

11396752

In [11]:
reg_data['author_citation_log'] = reg_data['author_citation'].apply(lambda x: np.log2(x+1))

In [12]:
reg_data['gender_CT'] = reg_data['gender'].apply(lambda gen: 1 if gen == 'Female' else 0)

In [13]:
co_feats_ = ['authorship_pos', 'author_pub_count_cate', 'affiliation_rank_cate', 'affiliation_cate', 'num_authors', \
         'journal_impact', 'author_citation_log', 'pub_year', \
        'Social_Sciences', 'Materials_Science', 'Engineering', 'Chemistry',
       'Biochemistry__Genetics_and_Molecular_Biology', 'Medicine', 'Nursing',
       'Agricultural_and_Biological_Sciences',
       'Pharmacology__Toxicology_and_Pharmaceutics', 'Neuroscience',
       'Business__Management_and_Accounting',
       'Economics__Econometrics_and_Finance', 'Chemical_Engineering',
       'Physics_and_Astronomy', 'Computer_Science', 'Decision_Sciences',
       'Health_Professions', 'Psychology', 'Immunology_and_Microbiology',
       'Dentistry', 'Earth_and_Planetary_Sciences', 'Environmental_Science',
       'Mathematics', 'Arts_and_Humanities', 'Energy', 'Veterinary', 'General']

In [15]:
X = dmatrix(formula_like=' + '.join(co_feats_), data=reg_data, return_type="dataframe")

In [16]:
list(X.columns)

['Intercept',
 'authorship_pos[T.last_position]',
 'authorship_pos[T.middle_position]',
 'authorship_pos[T.solo_author]',
 'affiliation_cate[T.international]',
 'pub_year[T.2014]',
 'pub_year[T.2015]',
 'pub_year[T.2016]',
 'pub_year[T.2017]',
 'pub_year[T.2018]',
 'author_pub_count_cate',
 'affiliation_rank_cate',
 'num_authors',
 'journal_impact',
 'author_citation_log',
 'Social_Sciences',
 'Materials_Science',
 'Engineering',
 'Chemistry',
 'Biochemistry__Genetics_and_Molecular_Biology',
 'Medicine',
 'Nursing',
 'Agricultural_and_Biological_Sciences',
 'Pharmacology__Toxicology_and_Pharmaceutics',
 'Neuroscience',
 'Business__Management_and_Accounting',
 'Economics__Econometrics_and_Finance',
 'Chemical_Engineering',
 'Physics_and_Astronomy',
 'Computer_Science',
 'Decision_Sciences',
 'Health_Professions',
 'Psychology',
 'Immunology_and_Microbiology',
 'Dentistry',
 'Earth_and_Planetary_Sciences',
 'Environmental_Science',
 'Mathematics',
 'Arts_and_Humanities',
 'Energy',
 'Vet

In [17]:
X['gender'] = reg_data['gender']
X['gender_CT'] = reg_data['gender_CT']
X['self_promotion'] = reg_data['self_promotion']

In [18]:
co_feats = [
 'authorship_pos[T.last_position]',
 'authorship_pos[T.middle_position]',
 'authorship_pos[T.solo_author]',
 'affiliation_cate[T.international]',
 'pub_year[T.2014]',
 'pub_year[T.2015]',
 'pub_year[T.2016]',
 'pub_year[T.2017]',
 'pub_year[T.2018]',
 'author_pub_count_cate',
 'affiliation_rank_cate',
 'num_authors',
 'journal_impact',
 'author_citation_log',
 'Social_Sciences',
 'Materials_Science',
 'Engineering',
 'Chemistry',
 'Biochemistry__Genetics_and_Molecular_Biology',
 'Medicine',
 'Nursing',
 'Agricultural_and_Biological_Sciences',
 'Pharmacology__Toxicology_and_Pharmaceutics',
 'Neuroscience',
 'Business__Management_and_Accounting',
 'Economics__Econometrics_and_Finance',
 'Chemical_Engineering',
 'Physics_and_Astronomy',
 'Computer_Science',
 'Decision_Sciences',
 'Health_Professions',
 'Psychology',
 'Immunology_and_Microbiology',
 'Dentistry',
 'Earth_and_Planetary_Sciences',
 'Environmental_Science',
 'Mathematics',
 'Arts_and_Humanities',
 'Energy',
 'Veterinary',
 'General']

In [19]:
# Y is the outcome, D is treatment status, and X is the independent variable
causal = CausalModel(Y=X['self_promotion'].values, D=X['gender_CT'].values, \
                     X=X[co_feats].values)

In [20]:
print(causal.summary_stats)


Summary Statistics

                   Controls (N_c=7371102)     Treated (N_t=4025650)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        0.046        0.210        0.035        0.183       -0.011

                   Controls (N_c=7371102)     Treated (N_t=4025650)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.192        0.394        0.135        0.342       -0.153
             X1        0.640        0.480        0.658        0.474        0.037
             X2        0.017        0.128        0.013        0.114       -0.030
             X3        0.653        0.476        0.650        0.477       -0.006
             X4        0.150        0.357        0.148        0.355       -0.006
      

In [21]:
causal.est_propensity()

In [49]:
# Automated propensity score estimation of which features to use in the matching.
# causal.est_propensity_s()

In [22]:
# Propensity model results
print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      0.225      0.003     75.843      0.000      0.219      0.231
            X0     -0.167      0.002    -71.758      0.000     -0.172     -0.162
            X1     -0.030      0.002    -16.818      0.000     -0.033     -0.026
            X2     -0.358      0.006    -62.505      0.000     -0.369     -0.347
            X3      0.048      0.001     32.877      0.000      0.045      0.050
            X4      0.033      0.002     13.401      0.000      0.028      0.038
            X5      0.068      0.002     28.586      0.000      0.063      0.072
            X6      0.100      0.002     42.848      0.000      0.096      0.105
            X7      0.127      0.002     54.302      0.000      0.122      0.131
            X8      0.188      0.002     79.389      0.000      0.

In [23]:
causal.propensity['fitted']

array([0.22737879, 0.42781661, 0.34181751, ..., 0.29336662, 0.44319718,
       0.54685937])

### 1 on 1 matching using the nearest neighbor

Check covariate balance before matching

In [24]:
for feat in co_feats:
    print(feat)
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_ind(X.loc[X['gender']=='Female', feat], X.loc[X['gender']=='Male', feat])[1])

authorship_pos[T.last_position]
	Female:	 0.13526237998832485
	Male:	 0.19153947401623259
	Diff:	 -0.05627709402790773
	T-test:	 0.0
authorship_pos[T.middle_position]
	Female:	 0.6580847813396594
	Male:	 0.6402958472152468
	Diff:	 0.017788934124412625
	T-test:	 0.0
authorship_pos[T.solo_author]
	Female:	 0.013133282823891795
	Male:	 0.016717581713019302
	Diff:	 -0.0035842988891275074
	T-test:	 0.0
affiliation_cate[T.international]
	Female:	 0.650354601120316
	Male:	 0.6530828090562306
	Diff:	 -0.0027282079359146616
	T-test:	 2.4062679946893132e-20
pub_year[T.2014]
	Female:	 0.14788270217232
	Male:	 0.14987067062699716
	Diff:	 -0.0019879684546771637
	T-test:	 2.1730750520364815e-19
pub_year[T.2015]
	Female:	 0.17185721560493336
	Male:	 0.1679662009832451
	Diff:	 0.0038910146216882546
	T-test:	 6.648496794265946e-63
pub_year[T.2016]
	Female:	 0.18450958230347894
	Male:	 0.17919735746432489
	Diff:	 0.005312224839154056
	T-test:	 9.030368977533984e-110
pub_year[T.2017]
	Female:	 0.18987716

In [25]:
len(causal.propensity['fitted'])

11396752

In [26]:
X['pscore'] = causal.propensity['fitted']

In [27]:
tem = X[['gender', 'pscore']].sort_values(by = ['pscore'])
tem['index'] = tem.index

In [28]:
tem = tem.values.tolist()

In [29]:
tem[-10:]

[['Female', 0.8572876306171883, 3463197],
 ['Female', 0.8575073542839112, 4991019],
 ['Male', 0.8580432050696255, 5813196],
 ['Male', 0.8587982301092821, 5556808],
 ['Female', 0.8593047742409025, 2470824],
 ['Female', 0.8593657656142847, 9668339],
 ['Female', 0.8593657656142847, 9668340],
 ['Female', 0.8593657656142847, 9668337],
 ['Female', 0.8593737653763763, 3176813],
 ['Female', 0.8610430114682025, 1008134]]

In [30]:
leng = len(tem)

In [31]:
pairs = {}

for i, elms in enumerate(tem):
    gen, score, ix = elms
    if gen == 'Female':
        pre_male_ix, nex_male_ix = 0, 0
        j = i-1
        while j >= 0:
            if tem[j][0] != 'Male':
                j -= 1
            else:
                break
        if j >= 0:
            pre_male_ix = j
        n = i+1
        while n <= leng-1:
            if tem[n][0] != 'Male':
                n += 1
            else:
                break
        if n <= leng-1:
            nex_male_ix = n
        if abs(score - tem[pre_male_ix][1]) <= abs(score - tem[nex_male_ix][1]):
            pairs[ix] = tem[pre_male_ix][2]
        else:
            pairs[ix] = tem[nex_male_ix][2]

In [32]:
del tem

In [33]:
len(pairs)

4025650

Check covariate balance after matching

In [34]:
for feat in co_feats:
    print('\n')
    print('Feat:', feat, '\n')
    print('Before matching:\n')
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_ind(X.loc[X['gender']=='Female', feat], X.loc[X['gender']=='Male', feat])[1])

    print('\nAfter matching:\n')
    f_avg = np.mean(X.loc[pairs.keys(), feat])
    m_avg = np.mean(X.loc[pairs.values(), feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_rel(X.loc[pairs.keys(), feat], X.loc[pairs.values(), feat])[1])



Feat: authorship_pos[T.last_position] 

Before matching:

	Female:	 0.13526237998832485
	Male:	 0.19153947401623259
	Diff:	 -0.05627709402790773
	T-test:	 0.0

After matching:

	Female:	 0.13526237998832485
	Male:	 0.13353868319401835
	Diff:	 0.0017236967943065062
	T-test:	 6.248364245614255e-14


Feat: authorship_pos[T.middle_position] 

Before matching:

	Female:	 0.6580847813396594
	Male:	 0.6402958472152468
	Diff:	 0.017788934124412625
	T-test:	 0.0

After matching:

	Female:	 0.6580847813396594
	Male:	 0.6613505893458199
	Diff:	 -0.0032658080061604977
	T-test:	 1.1390925041909596e-23


Feat: authorship_pos[T.solo_author] 

Before matching:

	Female:	 0.013133282823891795
	Male:	 0.016717581713019302
	Diff:	 -0.0035842988891275074
	T-test:	 0.0

After matching:

	Female:	 0.013133282823891795
	Male:	 0.013181722206351769
	Diff:	 -4.843938245997423e-05
	T-test:	 0.5414911717758715


Feat: affiliation_cate[T.international] 

Before matching:

	Female:	 0.650354601120316
	Male:	 0.6

	Female:	 0.014188267733161104
	Male:	 0.026809966813646047
	Diff:	 -0.012621699080484943
	T-test:	 0.0

After matching:

	Female:	 0.014188267733161104
	Male:	 0.015094456795796952
	Diff:	 -0.0009061890626358479
	T-test:	 7.081347405790692e-29


Feat: Decision_Sciences 

Before matching:

	Female:	 0.002887732415883149
	Male:	 0.004968320883363166
	Diff:	 -0.0020805884674800165
	T-test:	 0.0

After matching:

	Female:	 0.002887732415883149
	Male:	 0.0030330505632630753
	Diff:	 -0.00014531814737992615
	T-test:	 0.00012205663626571385


Feat: Health_Professions 

Before matching:

	Female:	 0.01080868927999205
	Male:	 0.01029045589112727
	Diff:	 0.0005182333888647805
	T-test:	 2.1376736710195383e-16

After matching:

	Female:	 0.01080868927999205
	Male:	 0.01072075317029548
	Diff:	 8.79361096965698e-05
	T-test:	 0.2181137076569987


Feat: Psychology 

Before matching:

	Female:	 0.039713835032851835
	Male:	 0.02157696366160718
	Diff:	 0.018136871371244655
	T-test:	 0.0

After matching:


In [58]:
for feat in co_feats:
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    f_avg_ = np.mean(X.loc[pairs.keys(), feat])
    m_avg_ = np.mean(X.loc[pairs.values(), feat])
    print(feat, ' & ', '{:6.2f}'.format(f_avg), ' & ', '{:6.2f}'.format(m_avg), ' & ', '{:6.2f}'.format(f_avg_), ' & ', '{:6.2f}'.format(m_avg_), ' \\\\ \hline')

authorship_pos[T.last_position]  &    0.14  &    0.19  &    0.14  &    0.13  \\ \hline
authorship_pos[T.middle_position]  &    0.66  &    0.64  &    0.66  &    0.66  \\ \hline
authorship_pos[T.solo_author]  &    0.01  &    0.02  &    0.01  &    0.01  \\ \hline
affiliation_cate[T.international]  &    0.65  &    0.65  &    0.65  &    0.65  \\ \hline
pub_year[T.2014]  &    0.15  &    0.15  &    0.15  &    0.15  \\ \hline
pub_year[T.2015]  &    0.17  &    0.17  &    0.17  &    0.17  \\ \hline
pub_year[T.2016]  &    0.18  &    0.18  &    0.18  &    0.18  \\ \hline
pub_year[T.2017]  &    0.19  &    0.18  &    0.19  &    0.19  \\ \hline
pub_year[T.2018]  &    0.18  &    0.19  &    0.18  &    0.18  \\ \hline
author_pub_count_cate  &    4.08  &    5.41  &    4.08  &    4.04  \\ \hline
affiliation_rank_cate  &    4.35  &    4.28  &    4.35  &    4.38  \\ \hline
num_authors  &   30.23  &   59.76  &   30.23  &   27.19  \\ \hline
journal_impact  &    5.23  &    5.59  &    5.23  &    5.25  \\ \hline

Avg treatment effect

In [35]:
# female avg
np.mean(X.loc[pairs.keys(), 'self_promotion'])

0.03473749580813036

In [36]:
# male avg
np.mean(X.loc[pairs.values(), 'self_promotion'])

0.045598598984015005

In [37]:
ttest_rel(X.loc[pairs.keys(), 'self_promotion'].apply(lambda x: 1 if x == True else 0), \
          X.loc[pairs.values(), 'self_promotion'].apply(lambda x: 1 if x == True else 0))

Ttest_relResult(statistic=-78.66416929505928, pvalue=0.0)

Avg treatment effect: `estimate via logit with covariates to control for the slight remaining imbalance`

In [38]:
reg_data['self_promotion'] = reg_data['self_promotion'].astype('int32')

In [ ]:
eq = "self_promotion ~ C(gender, Treatment(reference='Male')) + authorship_pos + author_pub_count_cate + \
    affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + author_citation_log + pub_year + \
        Social_Sciences + Materials_Science + Engineering + Chemistry + \
        Biochemistry__Genetics_and_Molecular_Biology + Medicine + Nursing + Agricultural_and_Biological_Sciences + \
        Pharmacology__Toxicology_and_Pharmaceutics + Neuroscience + Business__Management_and_Accounting + \
        Economics__Econometrics_and_Finance + Chemical_Engineering + Physics_and_Astronomy + Computer_Science + \
        Decision_Sciences + Health_Professions + Psychology + Immunology_and_Microbiology + Dentistry + \
        Earth_and_Planetary_Sciences + Environmental_Science + Mathematics + Arts_and_Humanities + Energy + \
        Veterinary + General"
model = smf.logit(formula = eq, data = reg_data.loc[list(pairs.keys()) + list(pairs.values())].sample(frac=1)).fit()

# for count DV.
# smf.glm(formula = eq, data=reg_data, family=sm.families.NegativeBinomial()).fit()

In [40]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         self_promotion   No. Observations:              8051300
Model:                          Logit   Df Residuals:                  8051257
Method:                           MLE   Df Model:                           42
Date:                Sun, 20 Nov 2022   Pseudo R-squ.:                 0.09350
Time:                        16:36:40   Log-Likelihood:            -1.2296e+06
converged:                       True   LL-Null:                   -1.3565e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================================================
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           -3.5594      0.010   -355.722      0.000      -3.579      -3.540
C(gender, Treatment(reference='Male'))[T.Female]    -0.3021      0.004    -81.613      0.000      -0.309      -0.295
authorship_pos[T.last_position]                     -0.3803      0.006    -66.384      0.000      -0.392      -0.369
authorship_pos[T.middle_position]                   -0.9775      0.004   -222.808      0.000      -0.986      -0.969
authorship_pos[T.solo_author]                        0.4408      0.010     42.393      0.000       0.420       0.461
affiliation_cate[T.international]                    0.1096      0.004     26.561      0.000       0.101       0.118
pub_year[T.2014]                                     0.3232      0.010     32.021      0.000       0.303       0.343
pub_year[T.2015]                                     0.6528      0.009     69.630      0.000       0.634       0.671
pub_year[T.2016]                                     0.8914      0.009     98.639      0.000       0.874       0.909
pub_year[T.2017]                                     1.2516      0.009    143.129      0.000       1.234       1.269
pub_year[T.2018]                                     1.6208      0.009    188.904      0.000       1.604       1.638
author_pub_count_cate                                0.0814      0.002     46.522      0.000       0.078       0.085
affiliation_rank_cate                               -0.0555      0.001    -78.812      0.000      -0.057      -0.054
num_authors                                         -0.0008   2.51e-05    -30.811      0.000      -0.001      -0.001
journal_impact                                       0.0352      0.000    150.641      0.000       0.035       0.036
author_citation_log                                 -0.0170      0.001    -16.118      0.000      -0.019      -0.015
Social_Sciences                                      0.8527      0.008    112.957      0.000       0.838       0.867
Materials_Science                                   -0.3331      0.018    -18.911      0.000      -0.368      -0.299
Engineering                                         -0.4449      0.015    -29.539      0.000      -0.474      -0.415
Chemistry                                           -0.6085      0.013    -47.162      0.000      -0.634      -0.583
Biochemistry__Genetics_and_Molecular_Biology        -0.2238      0.005    -43.397      0.000      -0.234      -0.214
Medicine                                            -0.3209      0.004    -76.017      0.000      -0.329      -0.313
Nursing                                              0.1953      0.010     18.635      0.000       0.175       0.216
Agricultural_and_Biological_Sciences                 0.5262      0.006     88.769      0.000       0.515       0.538
Pharmacology__Toxicology_and_Pharmaceutics          -0.6657      0.014    -47.529 

In [48]:
print(model.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                                    & self\_promotion  & \textbf{  No. Observations:  } &   8051300    \\
\textbf{Model:}                                            &      Logit       & \textbf{  Df Residuals:      } &   8051257    \\
\textbf{Method:}                                           &       MLE        & \textbf{  Df Model:          } &        42    \\
\textbf{Date:}                                             & Mon, 21 Nov 2022 & \textbf{  Pseudo R-squ.:     } &   0.09350    \\
\textbf{Time:}                                             &     09:00:05     & \textbf{  Log-Likelihood:    } & -1.2296e+06  \\
\textbf{converged:}                                        &       True       & \textbf{  LL-Null:           } & -1.3565e+06  \\
\textbf{Covariance Type:}                                  &    nonrobust     & \textbf{  LLR p-value:       } &     0.000    \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}


In [41]:
eq = "self_promotion ~ C(gender, Treatment(reference='Male'))"
model = smf.logit(formula = eq, data = reg_data.loc[list(pairs.keys()) + list(pairs.values())].sample(frac=1)).fit()

Optimization terminated successfully.
         Current function value: 0.168094
         Iterations 7


In [42]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         self_promotion   No. Observations:              8051300
Model:                          Logit   Df Residuals:                  8051298
Method:                           MLE   Df Model:                            1
Date:                Sun, 20 Nov 2022   Pseudo R-squ.:                0.002277
Time:                        16:38:45   Log-Likelihood:            -1.3534e+06
converged:                       True   LL-Null:                   -1.3565e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================================================
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           -3.0412      0.002  -1272.932      0.000      -3.046      -3.037
C(gender, Treatment(reference='Male'))[T.Female]    -0.2834      0.004    -78.245      0.000      -0.290      -0.276
====================================================================================================================
"""

In [49]:
print(model.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                                   & self\_promotion  & \textbf{  No. Observations:  } &   8051300    \\
\textbf{Model:}                                           &      Logit       & \textbf{  Df Residuals:      } &   8051298    \\
\textbf{Method:}                                          &       MLE        & \textbf{  Df Model:          } &         1    \\
\textbf{Date:}                                            & Sun, 20 Nov 2022 & \textbf{  Pseudo R-squ.:     } &   0.002277   \\
\textbf{Time:}                                            &     16:43:45     & \textbf{  Log-Likelihood:    } & -1.3534e+06  \\
\textbf{converged:}                                       &       True       & \textbf{  LL-Null:           } & -1.3565e+06  \\
\textbf{Covariance Type:}                                 &    nonrobust     & \textbf{  LLR p-value:       } &     0.000    \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
       